In [6]:
# libraries
import spacy
import unicodedata
from textblob import TextBlob
#from googletrans import Translator
from deep_translator import GoogleTranslator

import emoji
nlp = spacy.load('en_core_web_md')
translator = GoogleTranslator(source='auto', target='en')

import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS
import matplotlib.pyplot as plt

import time # for testing function speed
import numpy as np
from datetime import datetime
from deep_translator import GoogleTranslator
import re
import textfeatures as tf
import swifter
from langdetect import detect, LangDetectException, detect_langs


In [64]:
# load data and run function
path='C:/Users/alexd/Documents/ukraine_war_analysis/twitter/20230201_UkraineCombinedTweetsDeduped.csv.gzip'
data0 = pd.read_csv(path, compression='gzip')

data=pd.DataFrame(data=data0,columns=['username', 'text', 'totaltweets','followers', 'location', 'extractedts'])
#data['text'].dropna()
#data = data.head(10000)

In [67]:

start = time.time()
data = twitter_analysis(data, False)
end = time.time()
data.to_csv('DEMO_20230201.csv', header=True, index=False) 

print('time duration full process: ')
print(end - start)

time duration full process: 
173.31618666648865


In [66]:
def detect_language(text):
    try:
        if len(text) >= 10:
            return detect(text)
        else:
            return 'unknown'
    except LangDetectException:
        return 'unknown'

def twitter_analysis(data: pd.DataFrame, translate=False) -> pd.DataFrame:
    def extract_entities(text):
        # Extract entities from text
        doc = nlp(text)
        entities = {ent.label_: ent.text.lower() for ent in doc.ents if ent.label_ in ['GPE', 'PERSON']}
        return entities
    
    def clean_text(text):
        if translate:
            translator = GoogleTranslator(source='auto', target='en')
            text = translator.translate(text)
        text = re.sub(r'http\S+|#\w+|[^\w\s]|@\w+', '', text)  # remove URLs, hashtags, mentions, and punctuation
        text = re.sub('[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]', '', text)  # remove emojis
        text = text.strip().lower()  # convert to lowercase and remove leading/trailing whitespaces
        return text
    
    text = data['text'].apply(clean_text)
    is_english = text.apply(detect_language) == 'en'
    filtered_data = data[is_english & data['text'].str.contains('ukraine|russia|moscow|kiev|luhansk|sumy|chernihiv|zhytomyr|crimea', case=False)]
    dataset_output = (
        filtered_data.assign(text=text[is_english & data['text'].str.contains('ukraine|russia|moscow|kiev|luhansk|sumy|chernihiv|zhytomyr|crimea', case=False)])
            .assign(polarity=text.apply(lambda x: TextBlob(x).sentiment.polarity))
            .assign(**text.apply(extract_entities).apply(pd.Series, dtype='object'))
            .assign(extractedts=pd.to_datetime(filtered_data['extractedts']).dt.strftime('%d/%m/%Y'))
    )
    return dataset_output

In [49]:
path1='C:/Users/alexd/Documents/ukraine_war_analysis/DEMO_20230201.csv'
data1 = pd.read_csv(path1)
data1

,username,text,totaltweets,followers,location,extractedts,polarity,PERSON,GPE
0,1MedyaNews,i consider it absolutely logical that any furt...,5311,3192,"Kurdistan, Worldwide",02/01/2023,0.180000,sergey lavrov,NaN
1,queen_ukraine,ukraine war inside the battle for bakhmut,14540,46,NaN,02/01/2023,0.000000,NaN,ukraine
2,RealLoriSpencer,s refusal to facilitate inspection activities ...,52930,6181,USA,02/01/2023,0.400000,NaN,the united states
3,juliasskr,since for all their lives thats all russians w...,8067,2056,Kyiv Ukraine,02/01/2023,0.000000,NaN,NaN
4,medyanews_,i consider it absolutely logical that any furt...,14848,8153,"Kurdistan, Worldwide",02/01/2023,0.180000,sergey lavrov,NaN
5,ripeworldnews,british intelligence map as of jan 31\n\nin t...,1360,14,NaN,02/01/2023,0.120000,NaN,russia
6,intermarium24,the germans are not slacking off bmp marder ap...,2681,5647,News from Baltic to Black Sea,02/01/2023,0.000000,NaN,ukraine
7,AliaVeraNews,the head of the federal air transport agency n...,2598,8,"United States, Europe, Asia",02/01/2023,-0.100000,NaN,NaN
8,BackupQuayle,greekgod7777 rbus39337410 against has been o...,7422,1758,"USA, USA, USA",02/01/2023,0.150000,trump,ukraine
9,AliaVeraNews,telegraph funds from the sale of chelsea will ...,2598,8,"United States, Europe, Asia",02/01/2023,0.000000,NaN,ukraine


In [62]:


def twitter_analysis(data: pd.DataFrame, translate=False) -> pd.DataFrame:
    def extract_entities(text):
        # Extract entities from text
        doc = nlp(text)
        entities = {ent.label_: ent.text.lower() for ent in doc.ents if ent.label_ in ['GPE', 'PERSON']}
        return entities
    
    def clean_text(text):
        if translate:
            translator = GoogleTranslator(source='auto', target='en')
            text = translator.translate(text)
        text = re.sub(r'http\S+|#\w+|[^\w\s]|@\w+', '', text)  # remove URLs, hashtags, mentions, and punctuation
        text = re.sub('[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]', '', text)  # remove emojis
        text = text.strip().lower()  # convert to lowercase and remove leading/trailing whitespaces
        return text
    
    text = data['text'].swifter.apply(clean_text)
    is_english = text.swifter.apply(detect_language) == 'en'
    filtered_data = data[is_english & data['text'].str.contains('ukraine|russia|moscow|kiev|luhansk|sumy|chernihiv|zhytomyr|crimea', case=False)]
    dataset_output = (
        filtered_data.assign(text=text[is_english & data['text'].str.contains('ukraine|russia|moscow|kiev|luhansk|sumy|chernihiv|zhytomyr|crimea', case=False)])
            .assign(polarity=text.swifter.apply(lambda x: TextBlob(x).sentiment.polarity))
            .assign(**text.swifter.apply(extract_entities).swifter.apply(pd.Series, dtype='object'))
            .assign(extractedts=pd.to_datetime(filtered_data['extractedts']).dt.strftime('%d/%m/%Y'))
    )
    return dataset_output
